In [1]:
import tensorflow as tf
import os

# Set TensorFlow to use GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Force GPU 0
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)  # Prevents memory overflow
        print("✅ TensorFlow is now using the GPU!")
    except RuntimeError as e:
        print(e)


In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

In [3]:
base_model = ResNet50(weights="imagenet",include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 8s 0us/step


In [4]:
base_model.trainable = False

In [5]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(8, activation="softmax")(x)

In [6]:
model = Model(inputs=base_model.input, outputs=x)

In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "../fer2013plus/fer2013/train"
test_dir = "../fer2013plus/fer2013/test"

IMG_SIZE = (224, 224)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode="sparse",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode="sparse",
    subset="validation" 
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode="sparse",
    shuffle=False
)

Found 25551 images belonging to 8 classes.
Found 2835 images belonging to 8 classes.
Found 7099 images belonging to 8 classes.


In [15]:
# Print class labels
print("Class Labels:", train_generator.class_indices)

# Check number of images per class
import os
import numpy as np

def count_images_in_folders(directory):
    class_counts = {}
    for emotion_class in os.listdir(directory):
        class_path = os.path.join(directory, emotion_class)
        if os.path.isdir(class_path):
            class_counts[emotion_class] = len(os.listdir(class_path))
    return class_counts

# Print counts for train and test sets
train_counts = count_images_in_folders(train_dir)
test_counts = count_images_in_folders(test_dir)

print("Training Data Distribution:", train_counts)
print("Test Data Distribution:", test_counts)

Class Labels: {'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happiness': 4, 'neutral': 5, 'sadness': 6, 'surprise': 7}
Training Data Distribution: {'anger': 2466, 'contempt': 165, 'disgust': 191, 'fear': 652, 'happiness': 7528, 'neutral': 10308, 'sadness': 3514, 'surprise': 3562}
Test Data Distribution: {'anger': 644, 'contempt': 51, 'disgust': 57, 'fear': 167, 'happiness': 1827, 'neutral': 2597, 'sadness': 856, 'surprise': 900}


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get labels from the train generator
train_labels = train_generator.classes  # This contains all training labels

# Compute class weights
class_weights = compute_class_weight("balanced", classes=np.unique(train_labels), y=train_labels)

# Convert to dictionary format for Keras
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

print("Class Weights:", class_weights_dict)

Class Weights: {0: 1.4386824324324323, 1: 21.43540268456376, 2: 18.569040697674417, 3: 5.4410136286201025, 4: 0.4713510920897285, 5: 0.3442417546885105, 6: 1.00976130256086, 7: 0.9962180286961946}


In [17]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    class_weight=class_weights_dict
)

Epoch 1/30
 42/799 [>.............................] - ETA: 23:36 - loss: 2.4367 - accuracy: 0.0856

KeyboardInterrupt: 